# Understanding optical elements

We will present the internal workings of optical elements, and show how you can create your own optical elements in HCIPy.

We'll start by importing all the necessary packages.

In [ ]:
from hcipy import *
import numpy as np
import matplotlib.pyplot as plt

An `OpticalElement` is an object that can propagate a `Wavefront` from one plane to another. This includes apodizers, deformable mirrors, focal-plane masks, coronagraphs or even complete optical systems. An optical element should in principle be able to handle any type of `Wavefront` passed to its `forward()` and `backward()` functions. This puts a large responsibility on the implementation of the optical element.

Any proper implementation of an optical element should therefore implement a number of functions to help signal its intent. Here, we are going to implement a very simple `OpticalElement`: a neutral-density filter. We'll show the full implementation first, and then go in to the details for each implemented function.

In [ ]:
class NeutralDensityFilter(OpticalElement):
    def __init__(self, transmittance):
        self.transmittance = transmittance
    
    def forward(self, wavefront):
        wf = wavefront.copy()
        
        wf.electric_field *= np.sqrt(self.transmittance)
        
        return wf

    def backward(self, wavefront):
        wf = Wavefront.copy()
        
        wf.electric_field *= np.sqrt(self.transmittance)
        
        return wf

A neutral-density filter is a filter that reduces the intensity of the transmitted light independent of wavelength or spatial position. Therefore, the above implementation can handle any incoming wavefront. The initializer of the class just stores the passed transmittance. In the initializer, usually we prefer to do as much computation as possible, to aliviate the burden on the `forward()` and `backward()` functions. 

The `forward()` function propagates the wavefront through the filter. This function first creates a copy of the wavefront to work on, then modifies its electric field and returns it. Any implementation of `forward()` should not attempt to operate on a wavefront in-place and should always return a new `Wavefront` object.

The `backward()` function might need some extra explanation. It does not implement the propagation in the opposite direction through the optical element, or even the inverse or pseudoinverse of the forward propagation, but rather the adjoint. In absense of non-linear effects, any optical element can be thought of as a linear operator on a complex Hilbert space. This Hilbert space is the space of all possible wavefronts, and an optical element acts on a wavefront to produce another wavefront in the same space. This means that we can also construct the Hermitan adjoint of this operator. The `backward()` function implements this Hermitian adjoint propagation. In some cases, ie. when energy is conserved in the optical element, the `backward()` function is the inverse of the `forward()` function, but *in general this is not the case*.

As the previous optical element had a transmittance independent of spatial position and wavelength, its implementation was quite short. Some optical elements might be dependent on the grid and wavelength of the incoming wavefront. Their implementation can often be way more complicated because of this, especially if the arguments can be a function of both. For that reason, HCIPy offers an `AgnosticOpticalElement` to simplify their implementation. Let's look at an example of one of these: the `Apodizer`, one of the simplest optical elements that uses an `AgnosticOpticalElement`. This optical element is a thin mask of which the apodization can be a function of spatial position (ie. a `Grid`) and/or wavelength. The implementation is listed below.

In [ ]:
class Apodizer(AgnosticOpticalElement):
    def __init__(self, apodization):
        self.apodization = apodization

        AgnosticOpticalElement.__init__(self, grid_dependent=True, wavelength_dependent=True)

    def make_instance(self, instance_data, input_grid, output_grid, wavelength):
        instance_data.apodization = self.evaluate_parameter(self.apodization, input_grid, output_grid, wavelength)

    def get_input_grid(self, output_grid, wavelength):
        return output_grid

    def get_output_grid(self, input_grid, wavelength):
        return input_grid

    @make_agnostic_forward
    def forward(self, instance_data, wavefront):
        wf = wavefront.copy()

        wf.electric_field *= instance_data.apodization

        return wf

    @make_agnostic_backward
    def backward(self, instance_data, wavefront):
        wf = wavefront.copy()

        wf.electric_field *= np.conj(instance_data.apodization)

        return wf